In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [20]:
#Load ANN trained model,scaler,encoder
model = load_model('model.h5')
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
with open('ohe_encoder_gender.pkl','rb') as file:
    ohe_encoder_gender = pickle.load(file)
with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [21]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [22]:
geo_encoded = ohe_encoder_gender.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=ohe_encoder_gender.get_feature_names_out(['Geography']))
geo_encoded_df

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [23]:
input_df = pd.DataFrame([input_data])

In [24]:
input_df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [29]:
input_df['Gender']=label_encoder_gender.fit_transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,0,40,3,60000,2,1,1,50000


In [30]:
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [31]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,0,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [38]:
#Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516, -1.09499335,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [39]:
prediction= model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array([[0.05478996]], dtype=float32)

In [40]:
predic_prob = prediction[0][0]
predic_prob

0.054789964

In [41]:
if predic_prob > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
